# Modelling
<u>Tests using the following models :</u>
* Linear regression
* Random forest regressor
* Ridge and Lasso Regularization (add on to linear modelling?)

<u> Tests using the following variables:</u>
* Weather variables (rain, temperature, windspeed)
* Time variables (Day of week, month, year, time of day, public holiday)
* Sensor environment variables:
    * Sensor_id
    * Betweenness of the street 
    * Buildings in proximity to the sensor
    * Landmarks in proximity to the sensor  
    * Furniture in proximity to the sensor    
    * Lights in proximity to the sensor   


Normalise variables: should this be with MinMax or StandardScaler??

In [8]:
import copy
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, mean_squared_error,r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import time as thetime
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier, XGBRegressor

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

import multiprocessing

# To display tables in HTML output
from IPython.display import HTML, display

from Functions import *

### Read in formatted data

In [2]:
data = pd.read_csv("formatted_data_for_modelling.csv", index_col = False)

### Keep only sensors with relatively complete data

In [3]:
### Filter to include just sensors which we know have quite complete data 
data = data[data['sensor_id'].isin([2,6,9,10,14,18])]
data.reset_index(inplace=True, drop = True)

In [4]:
data = data.drop(['Pressure', 'Humidity'],axis=1) # seem obviously irrelevant
# data = data[data.columns.drop(list(data.filter(regex='h_')))] # (as these are replaced by the categorical ones)
data = data.drop(['sensor_id'],axis=1) # don't want this included
# Get rid of columns in which none of the sensors have a value
for column in data.columns:
    if np.nanmax(data[column]) ==0:
        del data[column]

In [5]:
df_subtypes = data.drop(['buildings', 'landmarks', 'furniture'],axis=1)

## Prepare data for modelling - split into predictor/predictand variables

In [6]:
# The predictor variables
Xfull = df_subtypes.drop(['hourly_counts'], axis =1)

# The variable to be predicted
Yfull = df_subtypes['hourly_counts'].values

# Split data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(Xfull, Yfull, test_size=0.6666, random_state=123)

#### Standardize both training and testing data
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# 
feature_list = list(Xfull.columns)

### Define models

In [11]:
# Linear regression
lr_model = LinearRegression()
# lr_model.fit(X_train, Y_train)

# Random forest (1000 decision trees)
rf_model = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# rf_model.fit(X_train, Y_train);

# XGB regressor
xgb_model = XGBRegressor()
# xgb_model.fit(X_train, Y_train);

### Cross-validation

In [13]:
def run_model_with_cv(model, metrics, cv):
  
    scores = cross_validate(model, Xfull, Yfull, cv=cv, scoring=metrics ,return_estimator=True)

    mae_scores = scores['test_neg_mean_absolute_error']
    r2_scores = scores['test_r2']
    rmse_scores = scores['test_neg_root_mean_squared_error']

    # Report the mean scores
    print("Mean mae of %0.2f with a standard deviation of %0.2f" % (mae_scores.mean(), mae_scores.std()))
    print("Mean r2 of %0.2f with a standard deviation of %0.2f" % (r2_scores.mean(), r2_scores.std()))
    print("Mean rmse of %0.2f with a standard deviation of %0.2f" % (rmse_scores.mean(), rmse_scores.std())) 

metrics = ['neg_mean_absolute_error', 'r2', 'neg_root_mean_squared_error']
cv = KFold(n_splits=10, random_state=1, shuffle=True)

run_model_with_cv(lr_model, metrics, cv)   

Mean mae of -260.30 with a standard deviation of 0.95
Mean r2 of 0.48 with a standard deviation of 0.00
Mean rmse of -354.47 with a standard deviation of 1.37


In [ ]:
run_model_with_cv(rf_model, metrics, cv)   

## Modelling - Linear regression

In [ ]:

## Set up a dictionary containing the hyperparameters we want to tune
hyperparameters_rf = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1],
                     'randomforestregressor__n_estimators': [50, 100, 150, 200, 250, 300, 350]}
# Set up the pipeline containing the scalers
pipeline_rf = make_pipeline(MinMaxScaler(feature_range = (0,1)), 
                         RandomForestRegressor())

# Store the scores in a results dictionary (and print them)
final_results = {}
for model_values in [(pipeline_rf,  hyperparameters_rf,  'RandomForest')]:
    
    clf = GridSearchCV(model_values[0], model_values[1], 
                       cv = 10, # positive intiger means k-fold (e.g. 10-fold)
                       #scoring  = 'neg_mean_squared_error', # MSE to calculate score
                       scoring  = 'r2', # MSE to calculate score
                       n_jobs=multiprocessing.cpu_count()) # Run on multiple cores
    clf.fit(X_test, Y_test)
    name = model_values[2]
    final_results[name] = clf
    print ("Hyperparameter results for {}".format(name))
    print ("\tBest Score: {}".format(clf.best_score_))
    print ("\tBest params: {}".format(clf.best_params_))

In [93]:
from sklearn.model_selection import cross_validate, cross_val_score




Mean mae of -130.77 with a standard deviation of 1.30
Mean r2 of 0.75 with a standard deviation of 0.00
Mean rmse of -243.62 with a standard deviation of 1.97


### Find the best model  
Use k-fold cross validation to evaluate a range of regression algorithms on the training data. Use a pipeline for evaluation which first scales the (weather) data. Print the results and assess which models perform best.

The following models were trialled:

* Decision Tree
* Random Forest
* Extra Trees
* Dummy Regressor
* Elastic Net CV
* Passive Aggressive
* RANSAC
* SGD
* TheilSen (dropped in code below because it takes too long)
* K Neighbours
* LinearRegression
* XGBoost

In [ ]:
# # Define a list of all the models to use
# Models = {'LinearRegression': LinearRegression,'DecisionTree' : DecisionTreeRegressor,
#           'RandomForest': RandomForestRegressor, 'ExtraTrees' : ExtraTreesRegressor,
#           'DummyRegressor' :DummyRegressor, 'ElasticNetCV' : ElasticNetCV, 
#           'PassiveAggressive' : PassiveAggressiveRegressor, #RANSAC': RANSACRegressor, # This one is terrible too
#           'SGD': SGDRegressor, #'TheilSen': TheilSenRegressor, # Drop this - it isn't great and takes too long
#           'KN': KNeighborsRegressor}#, 'XGBoost': xgb.XGBRegressor}
 
# # Now just run each model, but do this in multiple processes simultaneously to save time    
# # Now call that function simultaneously for each model
# p = Pool(processes=None) # A pool of processes (one for each core)
# results = p.map(run_model, [(name, model_type) for name, model_type in Models.items()])

# # Sort the results by median mse (that's item 5 in the tuple)
# results.sort(key=lambda x: x[5], reverse=True)

# # Put the results in a nice dictionary and print them
# results_dict = {}
# txt = "<table><thead><td>Name</td><td>Median R2</td><td>Median MSE</td><td>runtime (sec)</td></thead>"
# for name, model, all_r2, r2, all_mse, mse, runtime in results:
#     txt += "<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>".format(name, r2, mse, runtime)
#     results_dict[name] = (model, all_r2, r2, all_mse, mse, runtime)
# txt += "</table>"
# display(HTML(txt)) # print as html

# min_mse = min([mse for (name, model, all_r2, r2, all_mse, mse, runtime) in results])
               
# x =  [ name for (name, model, all_r2, r2, all_mse, mse, runtime) in results]
# y1 = [ mse-min_mse   for (name, model, all_r2, r2, all_mse, mse, runtime) in results]
# y2 = [ r2 if r2 > 0 else 0 for (name, model, all_r2, r2, all_mse, mse, runtime) in results]

# fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(15, 7))

# ax1.set_title("MSE")
# #ax1.invert_yaxis()
# ax1.bar(range(len(x)), y1)
# ax1.set_xticks(range(len(x)))
# ax1.set_xticklabels(x, rotation=90)
# ax1.set_ylim([27000000000, 29000000000])

# ax2.set_title("R^2")
# ax2.bar(range(len(x)), y2)
# ax2.set_xticks(range(len(x)))
# ax2.set_xticklabels(x, rotation=90)

# plt.show()

# #del x,y1, y2

# ## Set up a dictionary containing the hyperparameters we want to tune
# hyperparameters_rf = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
#                   'randomforestregressor__max_depth': [None, 5, 3, 1]}
# # hyperparameters_xgb = {'xgbregressor__max_depth': range(1, 11, 2),
# #                    'xgbregressor__n_estimators' : range(50, 400, 50),
# #                    'xgbregressor__learning_rate' : [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]}
# hyperparameters_lr = {}

# # Set up the pipeline containing the scalers
# pipeline_rf = make_pipeline(MinMaxScaler(feature_range = (0,1)), 
#                          RandomForestRegressor(n_estimators=100))
# # pipeline_xgb = make_pipeline(MinMaxScaler(feature_range = (0,1)),
# #                          xgb.XGBRegressor(n_estimators=100))
# pipeline_lr = make_pipeline(MinMaxScaler(feature_range = (0,1)),
#                          LinearRegression())

# # Store the scores in a results dictionary (and print them)
# final_results = {}
# for model_values in [(pipeline_rf,  hyperparameters_rf,  'RandomForest'),
# #                      (pipeline_xgb, hyperparameters_xgb, 'XGBoost'),
#                      (pipeline_lr,  hyperparameters_lr,  'LinearRegression')]:
    
#     clf = GridSearchCV(model_values[0], model_values[1], 
#                        #cv = None, # Cross-validation method. None means default (3-fold)
#                        cv = 10, # positive intiger means k-fold (e.g. 10-fold)
#                        #scoring  = 'neg_mean_squared_error', # MSE to calculate score
#                        scoring  = 'r2', # MSE to calculate score
#                        n_jobs=multiprocessing.cpu_count()) # Run on multiple cores
    
#     #clf = GridSearchCV(model_values[0], model_values[1], cv = 10, scoring  = 'r2')
#     clf.fit(X_validate, Y_validate)
#     name = model_values[2]
#     final_results[name] = clf
#     print ("Hyperparameter results for {}".format(name))
#     print ("\tBest Score: {}".format(clf.best_score_))
#     print ("\tBest params: {}".format(clf.best_params_))